In [1]:
import pandas as pd
import time
from tqdm import tqdm
tqdm.pandas()
time_start=time.time()

In [2]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])
answer_info=answer_info[['aid','qid','uid','atime']]
ques_info=ques_info[['qid','qtime']]

In [3]:
all_data=pd.concat([train[['qid','uid','time']],test])
all_data=all_data.merge(ques_info,on='qid',how='left')

In [4]:
len(all_data)

10630880

In [5]:
def time_(x):
    temp=x[1:].split('-H')
    return int(temp[0])*24+int(temp[1])

In [6]:
all_data.head()

,qid,uid,time,qtime
0,Q2166419046,M401693808,D3865-H22,D3861-H23
1,Q1550017551,M3392373099,D3844-H11,D3823-H20
2,Q604029601,M2317670257,D3862-H15,D3861-H15
3,Q2350061229,M1618461867,D3849-H11,D3847-H22
4,Q2443223942,M3544409350,D3867-H4,D3847-H15


In [7]:
#计算邀请时间
all_data['iday']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[0]))
all_data['ihour']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[1]))
all_data['itime']=all_data['time'].progress_apply(lambda x:time_(x))

100%|██████████| 10630880/10630880 [00:27<00:00, 389889.76it/s]


In [8]:
all_data['qday']=all_data['qtime'].progress_apply(lambda x:int(x[1:].split('-H')[0]))
all_data['qhour']=all_data['qtime'].progress_apply(lambda x:int(x[1:].split('-H')[1]))
all_data['qtime']=all_data['qtime'].progress_apply(lambda x:time_(x))

100%|██████████| 10630880/10630880 [00:24<00:00, 430484.80it/s]


In [9]:
all_data.head()

,qid,uid,time,qtime,iday,ihour,itime,qday,qhour
0,Q2166419046,M401693808,D3865-H22,92687,3865,22,92782,3861,23
1,Q1550017551,M3392373099,D3844-H11,91772,3844,11,92267,3823,20
2,Q604029601,M2317670257,D3862-H15,92679,3862,15,92703,3861,15
3,Q2350061229,M1618461867,D3849-H11,92350,3849,11,92387,3847,22
4,Q2443223942,M3544409350,D3867-H4,92343,3867,4,92812,3847,15


In [10]:
all_data['qweek']=all_data['qday'].progress_apply(lambda x:x%7)
all_data['iweek']=all_data['iday'].progress_apply(lambda x:x%7)


100%|██████████| 10630880/10630880 [00:14<00:00, 742339.95it/s]


In [11]:
all_data['itimeSqtime']=all_data['itime']-all_data['qtime']
all_data['idaySqday']=all_data['iday']-all_data['qday']

In [12]:
all_data.head()#iday  itime容易过拟合

,qid,uid,time,qtime,iday,ihour,itime,qday,qhour,qweek,iweek,itimeSqtime,idaySqday
0,Q2166419046,M401693808,D3865-H22,92687,3865,22,92782,3861,23,4,1,95,4
1,Q1550017551,M3392373099,D3844-H11,91772,3844,11,92267,3823,20,1,1,495,21
2,Q604029601,M2317670257,D3862-H15,92679,3862,15,92703,3861,15,4,5,24,1
3,Q2350061229,M1618461867,D3849-H11,92350,3849,11,92387,3847,22,4,6,37,2
4,Q2443223942,M3544409350,D3867-H4,92343,3867,4,92812,3847,15,4,3,469,20


In [13]:
#上次邀请时间，上次邀请是哪一天

In [14]:
all_data['q_invite_rank']=all_data.groupby(['qid'])['itime'].rank(ascending=1,method='min')#该问题第几次邀请

In [15]:
all_data['u_invite_rank']=all_data.groupby(['uid'])['itime'].rank(ascending=1,method='min')#邀请回答的第几个问题（区别第几次邀请）

In [16]:
all_data.head()

,qid,uid,time,qtime,iday,ihour,itime,qday,qhour,qweek,iweek,itimeSqtime,idaySqday,q_invite_rank,u_invite_rank
0,Q2166419046,M401693808,D3865-H22,92687,3865,22,92782,3861,23,4,1,95,4,29.0,3.0
1,Q1550017551,M3392373099,D3844-H11,91772,3844,11,92267,3823,20,1,1,495,21,5.0,3.0
2,Q604029601,M2317670257,D3862-H15,92679,3862,15,92703,3861,15,4,5,24,1,3.0,12.0
3,Q2350061229,M1618461867,D3849-H11,92350,3849,11,92387,3847,22,4,6,37,2,6.0,2.0
4,Q2443223942,M3544409350,D3867-H4,92343,3867,4,92812,3847,15,4,3,469,20,237.0,8.0


In [17]:
temp_all_data=all_data[['uid','itime']]

temp_all_data['rankid']=temp_all_data.groupby(['uid'])['itime'].rank(ascending=1,method='dense')

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
temp_all_data=temp_all_data.drop_duplicates(subset=['uid','rankid'])

In [19]:
temp_all_data

,uid,itime,rankid
0,M401693808,92782,3.0
1,M3392373099,92267,2.0
2,M2317670257,92703,12.0
3,M1618461867,92387,2.0
4,M3544409350,92812,5.0
5,M2818659842,92200,1.0
6,M2818659842,92684,8.0
7,M1581217469,92408,7.0
8,M3021021791,92419,3.0
9,M1766315480,92151,1.0


In [20]:
all_data['u_inviteNo_time']=all_data.groupby(['uid'])['itime'].rank(ascending=1,method='dense')#第几次邀请(根据总时间)

In [21]:
temp_all_data.columns=['uid','itime_last','u_inviteNo_time']
temp_all_data['u_inviteNo_time']=temp_all_data['u_inviteNo_time']+1

In [22]:
all_data=all_data.merge(temp_all_data,how='left',on=['uid','u_inviteNo_time'])
len(all_data)

10630880

In [23]:
all_data['inviteSlasttime']=all_data['itime']-all_data['itime_last']

In [24]:
all_data['inviteSlasttime']=all_data['inviteSlasttime'].fillna(0)

In [25]:
temp_all_data=all_data[['uid','iday']]
temp_all_data['rankid']=temp_all_data.groupby(['uid'])['iday'].rank(ascending=1,method='dense')
temp_all_data=temp_all_data.drop_duplicates(subset=['uid','rankid'])
temp_all_data.columns=['uid','iday_last','u_inviteNo_day']
temp_all_data['u_inviteNo_day']=temp_all_data['u_inviteNo_day']+1
all_data['u_inviteNo_day']=all_data.groupby(['uid'])['iday'].rank(ascending=1,method='dense')#第几次邀请(根据总时间)
all_data=all_data.merge(temp_all_data,how='left',on=['uid','u_inviteNo_day'])
len(all_data)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


10630880

In [26]:
all_data['inviteSlastday']=all_data['iday']-all_data['iday_last']
all_data['inviteSlastday']=all_data['inviteSlastday'].fillna(0)

In [27]:
all_data.columns

Index(['qid', 'uid', 'time', 'qtime', 'iday', 'ihour', 'itime', 'qday',
       'qhour', 'qweek', 'iweek', 'itimeSqtime', 'idaySqday', 'q_invite_rank',
       'u_invite_rank', 'u_inviteNo_time', 'itime_last', 'inviteSlasttime',
       'u_inviteNo_day', 'iday_last', 'inviteSlastday'],
      dtype='object')

In [28]:
all_data.drop(['iday_last', 'itime_last'],axis=1,inplace=True)

In [29]:
all_data

,qid,uid,time,qtime,iday,ihour,itime,qday,qhour,qweek,iweek,itimeSqtime,idaySqday,q_invite_rank,u_invite_rank,u_inviteNo_time,inviteSlasttime,u_inviteNo_day,inviteSlastday
0,Q2166419046,M401693808,D3865-H22,92687,3865,22,92782,3861,23,4,1,95,4,29.0,3.0,3.0,144.0,3.0,6.0
1,Q1550017551,M3392373099,D3844-H11,91772,3844,11,92267,3823,20,1,1,495,21,5.0,3.0,2.0,45.0,2.0,2.0
2,Q604029601,M2317670257,D3862-H15,92679,3862,15,92703,3861,15,4,5,24,1,3.0,12.0,12.0,50.0,11.0,2.0
3,Q2350061229,M1618461867,D3849-H11,92350,3849,11,92387,3847,22,4,6,37,2,6.0,2.0,2.0,59.0,2.0,2.0
4,Q2443223942,M3544409350,D3867-H4,92343,3867,4,92812,3847,15,4,3,469,20,237.0,8.0,5.0,12.0,5.0,1.0
5,Q640765464,M2818659842,D3841-H16,92057,3841,16,92200,3835,17,6,5,143,6,2.0,1.0,1.0,0.0,1.0,0.0
6,Q795459266,M2818659842,D3861-H20,86820,3861,20,92684,3617,12,5,4,5864,244,7.0,9.0,8.0,12.0,7.0,1.0
7,Q190554387,M1581217469,D3850-H8,92406,3850,8,92408,3850,6,0,0,2,0,4.0,7.0,7.0,21.0,7.0,1.0
8,Q1958712851,M3021021791,D3850-H19,92389,3850,19,92419,3849,13,6,0,30,1,27.0,3.0,3.0,242.0,3.0,10.0
9,Q311993584,M1766315480,D3839-H15,92107,3839,15,92151,3837,19,1,3,44,2,10.0,1.0,1.0,0.0,1.0,0.0


In [30]:
temp_all_data=all_data[['qid','iday']]
temp_all_data['rankid']=temp_all_data.groupby(['qid'])['iday'].rank(ascending=1,method='dense')

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
temp_all_data.head()

,qid,iday,rankid
0,Q2166419046,3865,3.0
1,Q1550017551,3844,4.0
2,Q604029601,3862,1.0
3,Q2350061229,3849,3.0
4,Q2443223942,3867,17.0


In [32]:

temp_all_data=temp_all_data.drop_duplicates(subset=['qid','rankid'])
temp_all_data.columns=['qid','qiday_last','q_inviteNo_day']
temp_all_data['q_inviteNo_day']=temp_all_data['q_inviteNo_day']+1
all_data['q_inviteNo_day']=all_data.groupby(['qid'])['iday'].rank(ascending=1,method='dense')#第几次邀请(根据总时间)
all_data=all_data.merge(temp_all_data,how='left',on=['qid','q_inviteNo_day'])
len(all_data)

10630880

In [33]:
all_data['qinviteSlastday']=all_data['iday']-all_data['qiday_last']
all_data['qinviteSlastday']=all_data['qinviteSlastday'].fillna(0)

In [34]:
temp_all_data=all_data[['qid','itime']]
temp_all_data['rankid']=temp_all_data.groupby(['qid'])['itime'].rank(ascending=1,method='dense')

temp_all_data=temp_all_data.drop_duplicates(subset=['qid','rankid'])
temp_all_data.columns=['qid','qitime_last','q_inviteNo_time']
temp_all_data['q_inviteNo_time']=temp_all_data['q_inviteNo_time']+1
all_data['q_inviteNo_time']=all_data.groupby(['qid'])['itime'].rank(ascending=1,method='dense')#第几次邀请(根据总时间)
all_data=all_data.merge(temp_all_data,how='left',on=['qid','q_inviteNo_time'])
len(all_data)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


10630880

In [35]:
all_data['qinviteSlasttime']=all_data['itime']-all_data['qitime_last']
all_data['qinviteSlasttime']=all_data['qinviteSlasttime'].fillna(0)

In [36]:
all_data.drop(['qiday_last', 'qitime_last'],axis=1,inplace=True)

In [39]:
all_data.to_hdf('feature_invite_time_new.h5', key='data')

In [38]:
all_data

,qid,uid,time,qtime,iday,ihour,itime,qday,qhour,qweek,...,q_invite_rank,u_invite_rank,u_inviteNo_time,inviteSlasttime,u_inviteNo_day,inviteSlastday,q_inviteNo_day,qinviteSlastday,q_inviteNo_time,qinviteSlasttime
0,Q2166419046,M401693808,D3865-H22,92687,3865,22,92782,3861,23,4,...,29.0,3.0,3.0,144.0,3.0,6.0,3.0,1.0,6.0,3.0
1,Q1550017551,M3392373099,D3844-H11,91772,3844,11,92267,3823,20,1,...,5.0,3.0,2.0,45.0,2.0,2.0,4.0,1.0,5.0,3.0
2,Q604029601,M2317670257,D3862-H15,92679,3862,15,92703,3861,15,4,...,3.0,12.0,12.0,50.0,11.0,2.0,1.0,0.0,3.0,3.0
3,Q2350061229,M1618461867,D3849-H11,92350,3849,11,92387,3847,22,4,...,6.0,2.0,2.0,59.0,2.0,2.0,3.0,1.0,3.0,20.0
4,Q2443223942,M3544409350,D3867-H4,92343,3867,4,92812,3847,15,4,...,237.0,8.0,5.0,12.0,5.0,1.0,17.0,1.0,83.0,6.0
5,Q640765464,M2818659842,D3841-H16,92057,3841,16,92200,3835,17,6,...,2.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,2.0,75.0
6,Q795459266,M2818659842,D3861-H20,86820,3861,20,92684,3617,12,5,...,7.0,9.0,8.0,12.0,7.0,1.0,2.0,11.0,2.0,261.0
7,Q190554387,M1581217469,D3850-H8,92406,3850,8,92408,3850,6,0,...,4.0,7.0,7.0,21.0,7.0,1.0,1.0,0.0,3.0,1.0
8,Q1958712851,M3021021791,D3850-H19,92389,3850,19,92419,3849,13,6,...,27.0,3.0,3.0,242.0,3.0,10.0,2.0,1.0,4.0,23.0
9,Q311993584,M1766315480,D3839-H15,92107,3839,15,92151,3837,19,1,...,10.0,1.0,1.0,0.0,1.0,0.0,2.0,1.0,3.0,20.0
